# **Simple Scenario Example - Charleston, USA**

In this notebook we demonstrate the **workflow** on how you can build a simple **FloodAdapt scenario** in Charleston, USA, using the **API**.  

We prepared a database, which you can download from [**here**](link.com). If you worked with the [**Database-Builder notebook**](database_builder.ipynb) beforehand, you can use that database.


In this notebook we will cover the following steps:

1. Create a synthetic **event** - Rainfall
2. Create a **projection** - Sea level rise (SLR)
3. Create a **measure** and **strategy** - Seawall
4. Create and run a **scenario**
6. Investigate the **output**


## **Step 0**: Import packages

In [8]:
# Import packages
from datetime import datetime
from pathlib import Path
from flood_adapt import Settings
from flood_adapt.dbs_classes.database import Database
from flood_adapt import unit_system as us
from flood_adapt.flood_adapt import FloodAdapt
from flood_adapt.objects.events.synthetic import SyntheticEvent 
from flood_adapt.objects.measures.measures import (
    FloodWall,
    MeasureType,
    SelectionType,
    HazardMeasure
)
from flood_adapt.objects.forcing.forcing import (
    ForcingSource,
    ForcingType,
)
from flood_adapt.objects.measures.measures import Measure
from flood_adapt.objects.projections.projections import Projection, PhysicalProjection, SocioEconomicChange
from flood_adapt.objects.forcing.time_frame import (
    TimeFrame,
)
from flood_adapt.objects.forcing.timeseries import (
    ShapeType,
    TimeseriesFactory,
)
from flood_adapt.objects.strategies.strategies import Strategy
from flood_adapt.objects.scenarios.scenarios import Scenario
from flood_adapt.config.sfincs import RiverModel
from flood_adapt.objects.forcing.discharge import DischargeConstant
from flood_adapt.objects.forcing.forcing import ForcingType
from flood_adapt.objects.forcing.rainfall import RainfallConstant
from flood_adapt.objects.forcing.waterlevels import (
    SurgeModel,
    TideModel,
    WaterlevelSynthetic,
)
from flood_adapt.objects.forcing.wind import WindConstant

## **Step 1**: Getting started - Initiate the database

Let's start with initiating the database.

In [ ]:
# Define the full path to your database file
full_path = Path(r"C:\Users\rautenba\Local_FA_Databases\Database\charleston_full")

# Set up the settings for the database
Settings(DATABASE_ROOT=full_path.parent, DATABASE_NAME=full_path.name)

# Create the FloodAdapt instance
fa = FloodAdapt(Settings().database_path)

## **Step 2**: Events - Create a synthetic Event

In [ ]:
# Create a synthetic event object
attrs = SyntheticEvent(
        name="test_synthetic_nearshore",
        time=TimeFrame(
            start_time=datetime(2020, 1, 1),
            end_time=datetime(2020, 1, 2),
        ),
        forcings={
            ForcingType.WIND: [
                WindConstant(
                    speed=us.UnitfulVelocity(value=5, units=us.UnitTypesVelocity.mps),
                    direction=us.UnitfulDirection(
                        value=60, units=us.UnitTypesDirection.degrees
                    ),
                )
            ],
            ForcingType.RAINFALL: [
                RainfallConstant(
                    intensity=us.UnitfulIntensity(
                        value=20, units=us.UnitTypesIntensity.mm_hr
                    )
                )
            ],
            ForcingType.DISCHARGE: [
                DischargeConstant(
                    river=RiverModel(
                        name="cooper",
                        description="Cooper River",
                        x_coordinate=595546.3,
                        y_coordinate=3675590.6,
                        mean_discharge=us.UnitfulDischarge(
                            value=5000, units=us.UnitTypesDischarge.cfs
                        ),
                    ),
                    discharge=us.UnitfulDischarge(
                        value=5000, units=us.UnitTypesDischarge.cfs
                    ),
                )
            ],
            ForcingType.WATERLEVEL: [
                WaterlevelSynthetic(
                    surge=SurgeModel(
                        timeseries=TimeseriesFactory.from_args(
                            shape_type=ShapeType.triangle,
                            duration=us.UnitfulTime(
                                value=1, units=us.UnitTypesTime.days
                            ),
                            peak_time=us.UnitfulTime(
                                value=8, units=us.UnitTypesTime.hours
                            ),
                            peak_value=us.UnitfulLength(
                                value=1, units=us.UnitTypesLength.meters
                            ),
                        )
                    ),
                    tide=TideModel(
                        harmonic_amplitude=us.UnitfulLength(
                            value=1, units=us.UnitTypesLength.meters
                        ),
                        harmonic_period=us.UnitfulTime(
                            value=12.4, units=us.UnitTypesTime.hours
                        ),
                        harmonic_phase=us.UnitfulTime(
                            value=0, units=us.UnitTypesTime.hours
                        ),
                    ),
                )
            ],
        },
    )

# Create the event
synthetic = fa.create_event(attrs.model_dump(exclude_none=True))

# Save the event to the database
fa.save_event(synthetic) 

## **Step 3**: Projections - Create a projection

In [ ]:
SLR_2FT = Projection(
    name="SLR_2ft",
    physical_projection=PhysicalProjection(
        sea_level_rise=us.UnitfulLength(value=2, units=us.UnitTypesLength.feet),
        subsidence=us.UnitfulLength(value=1, units=us.UnitTypesLength.feet),
    ),
    socio_economic_change=SocioEconomicChange(),
)

fa.create_projection(SLR_2FT.model_dump(exclude_none=True))
fa.save_projection(SLR_2FT)

## **Step 4**: Measures & Strategies

### **Step 4a**: Measures - Create a measure

In [ ]:
attrs_measure = FloodWall(
        name= "test_seawall",
        description = "seawall",
        type = MeasureType.floodwall,
        selection_type = SelectionType.polygon,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\seawall\seawall.geojson"),
        elevation = us.UnitfulLength(value=12, units=us.UnitTypesLength.feet))

floodwall = fa.create_measure(attrs_measure.model_dump(exclude_none=True), type=attrs_measure.type)
fa.save_measure(floodwall)

### **Step 4b**: Strategies - Create a strategy

In [ ]:
attrs = Strategy(
        name= "test_strategy",
        description = "seawall",
        measures = [attrs_measure.name],
        )

strategy = fa.create_strategy(attrs.model_dump(exclude_none=True))
fa.save_strategy(strategy)

## **Step X**: Create and run a scenario

In [9]:
event_name = synthetic.name
projection_name = SLR_2FT.name
strategy_name = strategy.name

scenario = Scenario(
    name = "current_nearshore_no_measures_slr2",
    description = "current_nearshore_no_measures_slr",
    event =  event_name,
    projection =  projection_name,
    strategy = strategy_name,
)
scenario = fa.create_scenario(scenario.model_dump())
fa.save_scenario(scenario)

fa.run_scenario(scenario.name)

2025-05-07 01:40:39 PM - FloodAdapt.SfincsAdapter - WARNING - Failed to add event rainfall multiplier, no rainfall forcing found in the model.
2025-05-07 01:40:40 PM - FloodAdapt.SfincsAdapter - WARNING - Could not use height data from file due to missing `z` column or missing values therein. Using uniform height of 12.0 feet instead.
